# SST_grad preprocessing

This notebook shows how to generate the SST_grad_timeseries.nc file

In [3]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [4]:
from dask.distributed import Client

In [5]:
from utils import area,ccrs_land,add_patches
import datetime

In [6]:
c = Client()
c

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33965 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:41497 Dashboard: /proxy/33965/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [9]:
inputfiles = path='/g/data/ua8/NOAA_OISST/AVHRR/v2-1_modified/oisst_avhrr_v2-1_*.nc'
# inputfiles='path2OISST/*.nc'

In [10]:
dataset_SST = xr.open_mfdataset(inputfiles,parallel=True)

In [11]:
dataset_SST

,Array,Chunk
Bytes,59.58 GB,1.52 GB
Shape,"(14367, 1, 720, 1440)","(366, 1, 720, 1440)"
Count,120 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,59.58 GB,1.52 GB
Shape,"(14367, 1, 720, 1440)","(366, 1, 720, 1440)"
Count,120 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


Coarsen dataset to a 1 degree grid.

In [ ]:
SST_coarsen = dataset_SST.sst.coarsen({'lat':4,'lon':4}).mean().compute()

### Compute SST gradient magnitude:

\begin{equation}
|\nabla SST| = \sqrt{\left(\frac{\partial SST}{\partial x}\right)^2 + \left(\frac{\partial SST}{\partial y}\right)^2 }
\end{equation}

In [13]:
dy,dx = area(SST_coarsen.lat,SST_coarsen.lon,return_grid=True)

In [19]:
dSST_dx = SST_coarsen.differentiate('lon').squeeze() / dx
dSST_dy = SST_coarsen.differentiate('lat').squeeze() / dy[np.newaxis,:,np.newaxis]

In [20]:
SST_grad = np.sqrt(dSST_dx**2 + dSST_dy**2)

Rechunk dataset

In [24]:
SST_grad_rechunk = SST_grad.chunk({"lat": 100, "lon": 100,'time':365})

In [25]:
SST_grad_rechunk

,Array,Chunk
Bytes,59.58 GB,14.60 MB
Shape,"(14367, 720, 1440)","(365, 100, 100)"
Count,15202 Tasks,4800 Chunks
Type,float32,numpy.ndarray


Roll over 12 months:

In [54]:
SST_grad_rolled = SST_grad_coarsen.rolling(time=365,center=True).mean().compute() #This computation requires ~ 50 GB of RAM.

In [60]:
SST_grad_rolled = SST_grad_rolled.to_dataset(name="SST")

In [ ]:
SST_grad_rolled

### Add metadata

In [61]:
SST_grad_rolled.attrs['title'] = "Sea Surface Temperature (SST) gradients"
SST_grad_rolled.attrs['Description'] = """SST gradients computed from OISST-NOAA."""
SST_grad_rolled.attrs['Publication'] = "Dataset created for Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
SST_grad_rolled.attrs['Author'] = "Josué Martínez-Moreno"
SST_grad_rolled.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

SST_grad_rolled.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

units = [r"$^\circ C m^{-1}$"]
names = ["SST_grad"]
long_names = ["Sea Surface Temperature gradient magnitude"]

counter=0
    
SST_grad_rolled["SST"].attrs['units'] = units[counter]
SST_grad_rolled["SST"].attrs['name'] = names[counter]
SST_grad_rolled["SST"].attrs['long_name'] = long_names[counter]

SST_grad_rolled["SST"].attrs['missing_value'] = np.nan
SST_grad_rolled["SST"].attrs['valid_min'] = np.nanmin(SST_grad_rolled["SST"])
SST_grad_rolled["SST"].attrs['valid_max'] = np.nanmax(SST_grad_rolled["SST"])
SST_grad_rolled["SST"].attrs['valid_range'] = [np.nanmin(SST_grad_rolled["SST"]),np.nanmax(dataset["SST"])]


## Store netCDF

In [62]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in EKE_rolled.data_vars}

EKE_rolled.to_netcdf('../datasets/SST_grad_timeseries.nc', encoding=encoding)